<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span><ul class="toc-item"><li><span><a href="#Strictest---Abstract-Creator" data-toc-modified-id="Strictest---Abstract-Creator-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Strictest - Abstract Creator</a></span></li><li><span><a href="#Somewhat-strict---Concrete-Creator" data-toc-modified-id="Somewhat-strict---Concrete-Creator-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Somewhat strict - Concrete Creator</a></span></li><li><span><a href="#Not-at-all-strict---Static-Method-Call" data-toc-modified-id="Not-at-all-strict---Static-Method-Call-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Not at all strict - Static Method Call</a></span></li></ul></li><li><span><a href="#Examples" data-toc-modified-id="Examples-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Examples</a></span></li></ul></div>

# Factory Pattern

## Intro

Factories are one of the most used patterns in Java. It's a creational pattern that handles the details needed when creating new objects. The point of the Factory is to remove the need to know how the constructor works of an object, making object creation a black box.

To implement the pattern, we need to use an interface to define how an object needs to work. This is called the Creator.

A key way to know when to use the Factory pattern is when a class needs to instantiate a subclass of a parent, but does not know which one. This allows the subclass to choose what to do.

A good real world example is a repair person going to repair a car. The repair person needs to know what car they are going to look at *ahead* of time to ensure they can actually do the job in question. They get this from you, the client.

The process of conducting the repair is done by the factory, in this case the repair person. But, you (the client) know that the repair is happening because you requested it.

Examples of where this is used in Java are:
* Java.util.Calendar
* NumberFormat
* valueOf method on the reference types

Advantages to using the pattern are:
* Decoupling of business logic from class logic (decouples production of objects from their use)
* Allows you to change design of the app without breaking anything
* Provides abstraction from concrete types

Disadvantages to using this pattern are:
* Clients may have to subeclass the Creator in order to access certain concrete types (not a major risk if you keep production of objs simplem but still a concern)
* Makes code difficult to read as it is heavily abstracted (constructors are a key way of understanding objs)
* Creating too many objects can hurt performance (garbage collector can help and singleton too)



## Implementation

### Strictest - Abstract Creator

There are 3 common implementations of the Factory pattern.

The strictest uses a Creator class as an abstract class. We'd then need to use the abstract class to create the individual product types. This then has a factory method as a member to create concrete objects from the class.

This uses delegation (the abstract class is the one doing the creating) and decouples the object creation from it's implementation.

In [9]:
interface Shape {
    void draw();
}

In [10]:
class Rectangle implements Shape {
    @Override
    public void draw()
    {
        System.out.println("Draw from rectangle");
    }
}

In the above, we have an interface (our Product) and a class created from it. If we wanted to create more rectangles *without* calling the constructor, it'd take more code than necessary. Also, if the implementation changed, the concrete class would break. Instead, we should use a factory to create concrete classes. 

In [11]:
abstract class ShapeFactory {
    protected abstract Shape createShape();
    
    public Shape getShape() {
        return createShape();
    }
}

In [12]:
class RectangleFactory extends ShapeFactory {
    protected Shape createShape()
    {
        return new Rectangle();
    }
}

In [18]:
Shape rectangle1 = new RectangleFactory().getShape();
rectangle1.draw();

Draw from rectangle


As we can see from the above, calling the getShape() method returns a shape instance. N.B. that the only thing we implement on the subclasses of the factory is the getShape(). This is because the parent class is responsible for the actual creation (more syntactic sugar as the subclass has access to the member due to it being protected. This was an intentional choice)

A downside to working in this way is that we'll need to make a new subclass for any new type of object we want to create. Not the worst thing, but if we have 500 different subclasses, this becomes extremely unwieldy.

### Somewhat strict - Concrete Creator

This approach is generally the way we implement the Factory Pattern in the real world. We use a single concrete creator class to handle object creation.

We add the implementation code to one factory method to create a product. We can switch on the types of objects by passing in an input param.

This allows us to not have to make a bajillion different subclasses of the abstract factory, mitigating a key risk and disadvantage.

Advantages of the pattern are:
* No need to implement multiple subclasses
* Clear how things work due to code living in one place

There aren't really any disadvantages to this approach. However, it's misuse can mislead clients and cause confusion over how objects work. Also, there is a need to implement various ways of creating objects, merely kicking the complexity can down the road.

In [19]:
interface Shape {
    void draw();
}

In [20]:
class Circle implements Shape {
    @Override
    public void draw()
    {
        System.out.println("Draw call from circle");
    }
}

In [22]:
class ShapeFactory {
    public Shape getShape(String shapeType)
    {
        switch(shapeType){
            case "circle":
                return new Circle();
            default:
                return null;
        }
    }
}

In [32]:
ShapeFactory shapeFactory = new ShapeFactory();
Shape circle1 = shapeFactory.getShape("circle");
circle1.draw();

Draw call from circle


As we can see from the above, it is already much easier to follow this approach to implementing the factory. We are using polymorphism to enforce a contract, but the factory instance is creating the right shape based on the input param.

The default at the end is flair I added to make the factory idempotent

In [33]:
Shape randomShape = shapeFactory.getShape("thing that isn't covered");
    
System.out.println(randomShape);

null


### Not at all strict - Static Method Call

This is the most lax way of implementing the Factory pattern. We do this by creating a static method on the factory. this allows us to keep the implementation of the factory *ringfenced* from the implementation of the concrete class due to scoping rules in Java.

However, this approach is rigid. You cannot subclass and change the behaviour of the create method, locking the implementation of the factory. This isn't always a bad thing, but is a concern.

In [39]:
interface Shape {
    void draw();
}

In [40]:
class Triangle implements Shape{
    @Override
    public void draw()
    {
        System.out.println("Draw call from Triangle");
    }
}

In [41]:
class ShapeFactory {
    public static Shape getShape(String shapeType)
    {
        switch(shapeType){
            case "triangle":
                return new Triangle();
            default:
                return null;
        }
    }
}

In [42]:
Shape triangle1 = ShapeFactory.getShape("triangle");
triangle1.draw();

Draw call from Triangle


This is the simplest way to implement a factory, but the one that scales the worst. The most strict scales best, but is a nightmare to maintain if it grows too big. The somewhat string is the best, all round approach. But, still has issues. Be mindful of the pitfalls of each.

## Examples

In [46]:
interface Animal {
    void walk();
    void eat();
}

In [48]:
class Duck implements Animal {
    
    @Override
    public void walk()
    {
        System.out.println("Duck is walking");
    }
    
    @Override
    public void eat()
    {
        System.out.println("Duck is eating");
    }
}

In [49]:
class Tiger implements Animal {
    @Override
    public void walk()
    {
        System.out.println("Tiger is walking");
    }
    
    @Override
    public void eat()
    {
        System.out.println("Tiger is eating");
    }
}

In [55]:
class AnimalFactory {
    public Animal createAnimal(String type)
    {
        switch(type.toLowerCase()){
            case "duck":
                return new Duck();
            case "tiger":
                return new Tiger();
            default:
                return null;
        }
    }
}

In [56]:
AnimalFactory animalFactory = new AnimalFactory();

Animal myTiger = animalFactory.createAnimal("tiger");
myTiger.walk();

Tiger is walking


In [57]:
Animal myDuck = animalFactory.createAnimal("DUCK");
myDuck.eat();

Duck is eating
